In [11]:
# --- put this at the very top of notebooks/mem0.ipynb ---
from mem0.memory.main import Memory as _Mem
import inspect as _inspect

# Only patch if needed (i.e., no "filters" in signature)
if "filters" not in str(_inspect.signature(_Mem.add)):
    _orig_add = _Mem.add
    def _add_with_filters(self, messages, *, user_id=None, agent_id=None, run_id=None,
                          metadata=None, infer=True, memory_type=None, prompt=None, filters=None):
        # pass everything through; ignore filters (proxy sends it)
        return _orig_add(self, messages,
                         user_id=user_id, agent_id=agent_id, run_id=run_id,
                         metadata=metadata, infer=infer, memory_type=memory_type, prompt=prompt)
    _Mem.add = _add_with_filters

In [5]:
from mem0.proxy.main import Mem0
import yaml
config = yaml.load(open("../config/mem0_config.yaml"), Loader=yaml.FullLoader)
client = Mem0(config=config)
r = client.chat.completions.create(
    # model="openrouter/openai/gpt-oss-20b",
    model="openai/gpt-5",
    messages=[{"role":"user","content":"I’m Alex, vegetarian, allergic to nuts."}],
    user_id="alex"
)
print(r.choices[0].message.content)

Exception in thread Thread-16 (add_task):
Traceback (most recent call last):
  File "/network/scratch/b/baldelld/hangman/venv/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/mila/b/baldelld/scratch/hangman/.venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "/network/scratch/b/baldelld/hangman/venv/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mila/b/baldelld/scratch/hangman/.venv/lib/python3.11/site-packages/mem0/proxy/main.py", line 155, in add_task
    self.mem0_client.add(
TypeError: Memory.add() got an unexpected keyword argument 'filters'


Nice to meet you, Alex! Thanks for letting me know you’re vegetarian and allergic to nuts—I’ll avoid nuts and nut products in anything I suggest.

A couple quick questions to tailor things:
- Do you eat eggs and/or dairy?
- How strict is the nut allergy (e.g., need to avoid cross-contact and nut oils)?
- Any other ingredients to avoid or cuisines you love?

I can help with:
- Nut-free vegetarian recipes or a simple meal plan
- Protein ideas (beans, lentils, chickpeas, peas, tofu/tempeh if soy is OK, seitan if gluten is OK, quinoa, eggs/dairy, seeds like sunflower/pumpkin if safe for you)
- Eating-out tips and label-checking guidance

How would you like to start?


In [13]:
from mem0 import Memory
m = Memory.from_config(config)

print("— get_all —")
for mm in m.get_all(user_id="alex")["results"]:
    print(mm["id"], "→", mm["memory"])

print("\n— search —")
for h in m.search("diet / allergy / vegetarian", user_id="alex", limit=5)["results"]:
    print(round(h["score"],3), "→", h["memory"])

— get_all —

— search —


In [2]:
# deps: mem0ai, openai (official SDK), pyyaml
# env: OPENROUTER_API_KEY, Qdrant running at 127.0.0.1:6333

import os, yaml, time
from collections import deque
from openai import OpenAI          # OpenRouter is OpenAI-compatible
from mem0 import Memory

# ---- Mem0 (OSS) config: Qdrant only matters here; LLM choice is for your responder (OpenRouter)
config = yaml.safe_load(open("../config/mem0_config.yaml"))
mem = Memory.from_config(config)

# ---- OpenRouter client
client = OpenAI(base_url="https://openrouter.ai/api/v1",
                api_key=os.environ["OPENROUTER_API_KEY"])

M_RECENT = 10   # paper m
S_SIM     = 10  # paper s
TOPK_GEN  = 5   # retrieval injected into responder prompt

def step_mem0(user_id: str, convo_window: deque, new_user_msg: str, model="openai/gpt-oss-20b"):
    """
    Paper-faithful loop:
    1) Extraction+Update: pass the *pair* (prev, current) + up to m-2 earlier messages to Memory.add(..., infer=True)
    2) Retrieval for answering: search top-k memories with current user query
    3) Response: include retrieved memories before generating an answer with OpenRouter
    """
    # 0) Build the turn-level messages as the paper uses a "pair" (m_{t-1}, m_t)
    # Keep a sliding window of the last M_RECENT messages in `convo_window`
    convo_window.append({"role": "user", "content": new_user_msg})
    while len(convo_window) > M_RECENT:
        convo_window.popleft()

    # 1) Extraction + Update (Mem0 base). This triggers candidate-fact extraction and ADD/UPDATE/DELETE/NOOP.  [oai_citation:3‡arXiv](https://arxiv.org/pdf/2504.19413)
    mem.add(list(convo_window), user_id=user_id, infer=True)

    # 2) Retrieval for answering (simple semantic query using current user msg)
    hits = mem.search(query=new_user_msg, user_id=user_id, limit=S_SIM)["results"]  # S_SIM is only for update in paper; for answer use TOPK_GEN below.  [oai_citation:4‡docs.mem0.ai](https://docs.mem0.ai/open-source/python-quickstart?utm_source=chatgpt.com)
    memtext = "\n".join(h["memory"] for h in hits[:TOPK_GEN]) if hits else "—"

    # 3) Respond with OpenRouter (inject retrieved memories)
    system = (
        "You are a helpful assistant.\n"
        "Known user facts (from long-term memory):\n"
        f"{memtext}\n\nUse these facts if relevant; do not invent new ones."
    )
    messages = [{"role": "system", "content": system}] + list(convo_window)

    resp = client.chat.completions.create(model=model, messages=messages)
    answer = resp.choices[0].message.content

    # After you send the assistant reply, append it so next *pair* is (user, assistant)
    convo_window.append({"role": "assistant", "content": answer})
    while len(convo_window) > M_RECENT:
        convo_window.popleft()
    return answer

# ---- one-shot example
window = deque([], maxlen=M_RECENT)
print(step_mem0("alex", window, "I’m Alex, vegetarian, allergic to nuts."))
time.sleep(1.0)  # allow DB write
print(step_mem0("alex", window, "Suggest dinner ideas in San Francisco."))

Hi Alex! I’ve noted that you’re vegetarian and have a nut allergy. How can I help you today? If you need dinner ideas, recipe tweaks, restaurant suggestions in San Francisco, or anything else, just let me know!
Hey Alex—happy to help you find a tasty, nut‑free vegetarian dinner in San Francisco! Below are a mix of restaurants and a quick home‑cook idea. I’ve double‑checked that each place is either strictly vegetarian/vegan (or offers vegetarian options) and that the kitchen staff can accommodate nut‑free needs. Remember to call ahead to confirm, especially if you’re still worried about cross‑contamination.

---

## 1. *The Plant Café* (Mission)

| Dish | Why it works | Nut‑free note |
|------|--------------|---------------|
| **Press‑Cooked Veggie Burger** | Made with lentils, veggies, and a homemade empanada patty. It’s a “soft” bite that’s easy to chew and can be swapped for the “classic” if you prefer. | Ask for a raw‑veggies side instead of any “nutty” sauces. |
| **Black Bean Tac

In [3]:
window

deque([{'role': 'user', 'content': 'I’m Alex, vegetarian, allergic to nuts.'},
       {'role': 'assistant',
        'content': 'Hi Alex! I’ve noted that you’re vegetarian and have a nut allergy. How can I help you today? If you need dinner ideas, recipe tweaks, restaurant suggestions in San\u202fFrancisco, or anything else, just let me know!'},
       {'role': 'user', 'content': 'Suggest dinner ideas in San Francisco.'},
       {'role': 'assistant',
        'content': 'Hey Alex—happy to help you find a tasty, nut‑free vegetarian dinner in San\u202fFrancisco! Below are a mix of restaurants and a quick home‑cook idea. I’ve double‑checked that each place is either strictly vegetarian/vegan (or offers vegetarian options) and that the kitchen staff can accommodate nut‑free needs. Remember to call ahead to confirm, especially if you’re still worried about cross‑contamination.\n\n---\n\n## 1. *The Plant Café* (Mission)\n\n| Dish | Why it works | Nut‑free note |\n|------|--------------|-------

In [9]:
from mem0 import Memory
m = Memory.from_config(config)

print("— get_all —")
for mm in mem.get_all(user_id="alex")["results"]:
    print(mm["id"], "→", mm["memory"])

print("\n— search —")
for h in mem.search("diet / allergy / vegetarian", user_id="alex", limit=5)["results"]:
    print(round(h["score"],3), "→", h["memory"])

— get_all —
466b1b71-ca45-4907-a585-907b8ff8d0c7 → Is a vegetarian
8a98cbdd-816e-4549-b2c2-30f67e609aa1 → Name is Alex
b90b046e-c733-4096-9f3b-faaf371948bc → Looking for dinner ideas in San Francisco
f2a3c4db-15f6-42cd-9940-c403503972b1 → Allergic to nuts

— search —
0.492 → Is a vegetarian
0.461 → Allergic to nuts
0.261 → Looking for dinner ideas in San Francisco
0.143 → Name is Alex


In [2]:
import mem0, inspect
from mem0.memory.main import Memory
print("mem0 version:", getattr(mem0, "__version__", "unknown"))
print("Memory.add signature:", inspect.signature(Memory.add))

mem0 version: 1.0.0b0
Memory.add signature: (self, messages, *, user_id: Optional[str] = None, agent_id: Optional[str] = None, run_id: Optional[str] = None, metadata: Optional[Dict[str, Any]] = None, infer: bool = True, memory_type: Optional[str] = None, prompt: Optional[str] = None)


In [7]:
PROMPT_TEMPLATE = """You are an intelligent memory assistant tasked with retrieving accurate information from
conversation memories.

# CONTEXT:
You have access to memories from two speakers in a conversation. These memories contain
timestamped information that may be relevant to answering the question.

# INSTRUCTIONS:
1. Carefully analyze all provided memories from both speakers
2. Pay special attention to the timestamps to determine the answer
3. If the question asks about a specific event or fact, look for direct evidence in the
memories
4. If the memories contain contradictory information, prioritize the most recent memory
5. If there is a question about time references (like "last year", "two months ago",
etc.), calculate the actual date based on the memory timestamp. For example, if a memory
from 4 May 2022 mentions "went to India last year," then the trip occurred in 2021.
6. Always convert relative time references to specific dates, months, or years. For
example, convert "last year" to "2022" or "two months ago" to "March 2023" based on the
memory timestamp. Ignore the reference while answering the question.
7. Focus only on the content of the memories from both speakers. Do not confuse character
names mentioned in memories with the actual users who created those memories.
8. The answer should be less than 5-6 words.

# APPROACH (Think step by step):
1. First, examine all memories that contain information related to the question
2. Examine the timestamps and content of these memories carefully
3. Look for explicit mentions of dates, times, locations, or events that answer the
question
4. If the answer requires calculation (e.g., converting relative time references), show
your work
5. Formulate a precise, concise answer based solely on the evidence in the memories
6. Double-check that your answer directly addresses the question asked
7. Ensure your final answer is specific and avoids vague time references

Memories for user {speaker_1_user_id}:
{speaker_1_memories}

Memories for user {speaker_2_user_id}:
{speaker_2_memories}

Question: {question}
Answer:
"""

In [12]:
from openai import OpenAI
from mem0 import Memory
from collections import deque
import os, yaml
import time

config = yaml.safe_load(open("../config/mem0_config.yaml"))
mem = Memory.from_config(config)
client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=os.environ["OPENROUTER_API_KEY"])

M_RECENT, S_SIM, TOPK_GEN = 10, 10, 5

def mem0_step(user_id: str, convo_window: deque, user_message: str, model="openai/gpt-oss-20b"):
    convo_window.append({"role": "user", "content": user_message})
    while len(convo_window) > M_RECENT:
        convo_window.popleft()

    # Extraction + Update
    mem.add(list(convo_window), user_id=user_id, infer=True)

    # Retrieve top-k memories (used as factual context)
    hits = mem.search(query=user_message, user_id=user_id, limit=S_SIM)["results"]
    memtext = "\n".join(h["memory"] for h in hits[:TOPK_GEN])

    # --- Faithful injection ---
    # As per paper: prepend retrieved facts *as is* before user message.
    messages = []
    if memtext.strip():
        messages.append({"role": "system", "content": f"Memories:\n{memtext}"})
    messages.extend(list(convo_window))  # previous turns + current user

    resp = client.chat.completions.create(model=model, messages=messages)
    answer = resp.choices[0].message.content
    convo_window.append({"role": "assistant", "content": answer})
    return answer

# ---- one-shot example
window = deque([], maxlen=M_RECENT)
print(mem0_step("alex", window, "I’m Alex, vegetarian, allergic to nuts."))
time.sleep(1.0)  # allow DB write
print(mem0_step("alex", window, "I am homosexual and specifically I like blowjobs with foots."))
time.sleep(1.0)
# print(mem0_step("alex", window, "Suggest dinner ideas in San Francisco."))
# print(mem0_step("alex", window, "I want to play hangman. You be the host. Think of a secret word and tell me the number of letters in it. I am going to try to guess it letter by letter. You will tell me if I am right or wrong."))

Nice to meet you, Alex! 🌱 If you’re looking for vegetarian dinner ideas around San Francisco that are nut‑free, I’ve got plenty of tasty options for you. Let me know what kind of cuisine you’re in the mood for (e.g., tacos, pasta, sushi, etc.) or if you’d like a recipe, restaurant recommendation, or a grocery list. I’m happy to help!
That’s a personal preference and totally valid—everyone’s sexuality is unique. If you’re looking to talk about how to explore it respectfully and safely with a partner (e.g., establishing clear boundaries, practicing good hygiene, or finding communities that share similar tastes), I’m here to help. Just let me know what specific information or guidance you’d like, and we’ll keep things respectful and non‑graphic.


In [13]:
print("— get_all —")
for mm in mem.get_all(user_id="alex")["results"]:
    print(mm["id"], "→", mm["memory"])

print("\n— search —")
for h in mem.search("diet / allergy / vegetarian", user_id="alex", limit=5)["results"]:
    print(round(h["score"],3), "→", h["memory"])

— get_all —
05e1ba70-be6c-43d1-8d82-6ee971b2834b → Is homosexual
2569fe86-90a8-4ed5-a127-79e27a4c7d95 → Likes blowjobs with feet
466b1b71-ca45-4907-a585-907b8ff8d0c7 → Is a vegetarian
8a98cbdd-816e-4549-b2c2-30f67e609aa1 → Name is Alex
b90b046e-c733-4096-9f3b-faaf371948bc → Looking for dinner ideas in San Francisco
f2a3c4db-15f6-42cd-9940-c403503972b1 → Allergic to nuts

— search —
0.492 → Is a vegetarian
0.461 → Allergic to nuts
0.261 → Looking for dinner ideas in San Francisco
0.166 → Is homosexual
0.143 → Name is Alex


In [11]:
# put this VERY early in your process (before Mem0/LiteLLM calls)
import litellm
from litellm import utils as _lu

_orig = _lu.supports_function_calling

_WHITELIST = {
    "openrouter/qwen/qwen3-32b",
    "openrouter/qwen/qwen3-235b-a22b-thinking-2507",
}

def _patched_supports_function_calling(model: str) -> bool:
    if model in _WHITELIST:
        return True
    return _orig(model)

_lu.supports_function_calling = _patched_supports_function_calling

In [1]:
from hangman.agents.mem0_agent import Mem0Agent
from hangman.providers.llmprovider import load_llm_provider
from langchain_core.messages import HumanMessage
import os, yaml


CONFIG_PATH = "../config/config.yaml"          # LLM + Mem0/Qdrant config
MEM0_CONFIG_PATH = "../config/mem0_config_qwen3_235b.yaml"

print("Is file readable: ", os.access(CONFIG_PATH, os.R_OK))
with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

# Load your OpenRouter-backed LLMProvider
try:
    # e.g., "qwen3_235b_openrouter" or "gpt_oss_20b_openrouter" as you use in ReActMem
    main_llm = load_llm_provider(CONFIG_PATH, provider_name="qwen3_235b_openrouter")
    print("✅ LLM Provider loaded successfully.")
except Exception as e:
    print(f"❌ Failed to load LLM Provider: {e}")
    raise SystemExit(1)

# Initialize the Mem0Agent
agent = Mem0Agent(
    llm_provider=main_llm,
    mem0_config_path=MEM0_CONFIG_PATH,
    session_id="mem0_session_1",
    m_recent=10,       # paper: m=10
    s_neighbors=10,    # paper: s=10 (used internally by update; OSS may use default)
    k_per_user=10,     # retrieved memories per speaker for QA prompt
)
print("🤖 Mem0Agent is ready. Type 'quit', 'exit', or 'q' to end.")

# Interactive loop (same shape as ReActMem)


2025-10-20 12:39:22,003 - INFO - Created index for user_id in collection mem0
2025-10-20 12:39:22,009 - INFO - Created index for agent_id in collection mem0
2025-10-20 12:39:22,014 - INFO - Created index for run_id in collection mem0
2025-10-20 12:39:22,019 - INFO - Created index for actor_id in collection mem0


Is file readable:  True
✅ LLM Provider loaded successfully.


2025-10-20 12:39:22,082 - INFO - Created index for user_id in collection mem0migrations
2025-10-20 12:39:22,091 - INFO - Created index for agent_id in collection mem0migrations
2025-10-20 12:39:22,099 - INFO - Created index for run_id in collection mem0migrations
2025-10-20 12:39:22,105 - INFO - Created index for actor_id in collection mem0migrations


🤖 Mem0Agent is ready. Type 'quit', 'exit', or 'q' to end.


In [ ]:
# # put this VERY early in your process (before Mem0/LiteLLM calls)
# import litellm
# from litellm import utils as _lu

# _orig = _lu.supports_function_calling

# _WHITELIST = {
#     "openrouter/qwen/qwen3-32b",
#     "openrouter/qwen/qwen3-235b-a22b-thinking-2507",
# }

# def _patched_supports_function_calling(model: str) -> bool:
#     if model in _WHITELIST:
#         return True
#     return _orig(model)

# _lu.supports_function_calling = _patched_supports_function_calling

messages = [HumanMessage(content="I’m Alex, vegetarian, allergic to nuts.")]
agent.invoke(messages)

12:39:22 - LiteLLM:INFO: utils.py:3373 - 
LiteLLM completion() model= qwen/qwen3-235b-a22b-thinking-2507; provider = openrouter
2025-10-20 12:39:22,186 - INFO - 
LiteLLM completion() model= qwen/qwen3-235b-a22b-thinking-2507; provider = openrouter


/home/mila/b/baldelld/scratch/hangman/.venv/lib/python3.11/site-packages/httpx/_models.py:408: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  headers, stream = encode_request(
12:40:01 - LiteLLM:INFO: utils.py:1286 - Wrapper: Completed Call, calling success_handler
2025-10-20 12:40:01,101 - INFO - Wrapper: Completed Call, calling success_handler
2025-10-20 12:40:05,281 - INFO - Total existing memories: 3
12:40:05 - LiteLLM:INFO: utils.py:3373 - 
LiteLLM completion() model= qwen/qwen3-235b-a22b-thinking-2507; provider = openrouter
2025-10-20 12:40:05,282 - INFO - 
LiteLLM completion() model= qwen/qwen3-235b-a22b-thinking-2507; provider = openrouter


In [6]:
import litellm
from litellm import model_cost, model_alias_map

model_name = "openrouter/qwen/qwen3-235b-a22b-thinking-2507"

for model_name in ["openrouter/qwen/qwen3-235b-a22b-thinking-2507", "openrouter/qwen/qwen3-32b", "openrouter/openai/gpt-oss-120b", "openrouter/qwen/qwen3-235b-a22b-thinking-2507"]:   
    # 1️⃣ Check whether the model is in LiteLLM’s registry
    if model_name in model_cost or model_name in model_alias_map:
        print(f"{model_name} exists in LiteLLM registry.")
    else:
        print(f"{model_name} NOT found in LiteLLM’s internal model list.")

    # 2️⃣ Then, check if LiteLLM believes it supports function calling
    from litellm.utils import supports_function_calling

    print("supports_function_calling:", supports_function_calling(model_name))

openrouter/qwen/qwen3-235b-a22b-thinking-2507 NOT found in LiteLLM’s internal model list.
supports_function_calling: False
openrouter/qwen/qwen3-32b NOT found in LiteLLM’s internal model list.
supports_function_calling: False
openrouter/openai/gpt-oss-120b exists in LiteLLM registry.
supports_function_calling: True
openrouter/qwen/qwen3-235b-a22b-thinking-2507 NOT found in LiteLLM’s internal model list.
supports_function_calling: False


In [5]:
from litellm import completion
response = completion(
            model="openrouter/qwen/qwen3-235b-a22b-thinking-2507",
            messages=[{"role": "user", "content": "write code for saying hi"}]
)
response

ModelResponse(id='gen-1760977175-xoAoVj6dz5wl2LmC6TKi', created=1760977175, model='qwen/qwen3-235b-a22b-thinking-2507', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='Here are simple code examples in several popular programming languages that output "Hi" to the console:\n\n---\n\n### **Python**\n```python\nprint("Hi")\n```\n\n---\n\n### **JavaScript** (Node.js or Browser Console)\n```javascript\nconsole.log("Hi");\n```\n\n---\n\n### **Java**\n```java\npublic class Main {\n    public static void main(String[] args) {\n        System.out.println("Hi");\n    }\n}\n```\n\n---\n\n### **C**\n```c\n#include <stdio.h>\n\nint main() {\n    printf("Hi\\n");\n    return 0;\n}\n```\n\n---\n\n### **C++**\n```cpp\n#include <iostream>\n\nint main() {\n    std::cout << "Hi" << std::endl;\n    return 0;\n}\n```\n\n---\n\n### **Bash**\n```bash\necho "Hi"\n```\n\n---\n\n### **Ruby**\n```ruby\nputs "Hi"\n```\n\n---\n\n### **C#**\

In [7]:
# run this once early, before Mem0 imports its LLM
from litellm import utils as _lu
_orig = _lu.supports_function_calling

WHITELIST = {
    "openrouter/qwen/qwen3-32b",
    "openrouter/qwen/qwen3-235b-a22b-thinking-2507",
}

def patched_supports(model: str) -> bool:
    if model in WHITELIST:
        return True
    return _orig(model)

_lu.supports_function_calling = patched_supports

import litellm
from litellm import model_cost

for m in [
    "openrouter/qwen/qwen3-32b",
    "openrouter/qwen/qwen3-235b-a22b-thinking-2507",
]:
    if m not in model_cost:
        model_cost[m] = {
            "input_cost_per_token": 0.0,   # or your real price if you care
            "output_cost_per_token": 0.0,
            "supports_function_calling": True,  # <-- makes the check pass
            "max_input_tokens": 131072,        # a safe default
        }

In [8]:
from litellm import completion

tools = [{
  "type": "function",
  "function": {
    "name": "echo",
    "description": "echo back a string",
    "parameters": {
      "type": "object",
      "properties": {"text": {"type": "string"}},
      "required": ["text"]
    }
  }
}]

r = completion(
    model="openrouter/qwen/qwen3-235b-a22b-thinking-2507",
    messages=[{"role":"user","content":"call echo('hi')"}],
    tools=tools,
    tool_choice="auto",
)
print(r.choices[0].message)  # look for tool_calls

Message(content='', role='assistant', tool_calls=[ChatCompletionMessageToolCall(index=0, function=Function(arguments='{"text": "hi"}', name='echo'), id='f97022ec7', type='function')], function_call=None, provider_specific_fields={'refusal': None, 'reasoning': None})
